In [1]:
import facebook
import os
import requests
import pandas as pd

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

# Directory di Staging
dir_df = os.path.join(os.path.abspath(''),'stg')

In [11]:
# Recupero tutti i posts di Salvini (ID e Data Creazione)
df_filename = r'df_posts_m5s.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df_posts = pd.read_pickle(df_fullpath)

In [12]:
df_posts.head(5)

,ID,Post_Date
0,174457180812_10151715791785813,2013-07-17T17:20:00+0000
1,174457180812_10152502294540813,2014-07-22T17:39:19+0000
2,174457180812_10152000804565813,2013-11-26T16:00:42+0000
3,174457180812_10152615088325813,2014-09-08T12:14:51+0000
4,174457180812_10152216213355813,2014-03-08T17:00:24+0000


In [13]:
#Esempio di get_likes
#graph.get_connections(id='252306033154_10151694974523155', connection_name='likes', summary='true')['summary']['total_count']

In [14]:
def get_likes(post):
    likes = None
    try:
        likes = graph.get_connections(id=post['ID'], connection_name='likes', summary='true')['summary']['total_count']
    except:
        likes = None
    return likes

In [15]:
# Molto Lento, impiega ca. 2 ore su 16000 post, va ottimizzato (thread, parallel, etc)
df_posts['Likes'] = df_posts.apply(lambda x: get_likes(x), axis=1)

In [16]:
df_posts.head(5)

,ID,Post_Date,Likes
0,174457180812_10151715791785813,2013-07-17T17:20:00+0000,411.0
1,174457180812_10152502294540813,2014-07-22T17:39:19+0000,57.0
2,174457180812_10152000804565813,2013-11-26T16:00:42+0000,599.0
3,174457180812_10152615088325813,2014-09-08T12:14:51+0000,43.0
4,174457180812_10152216213355813,2014-03-08T17:00:24+0000,111.0


In [17]:
# Salvo il DF per non ripetere la Chiamata
df_filename = r'df_posts_likes_m5s.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df_posts.to_pickle(df_fullpath)